In [53]:
#Librerias

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import a_funciones as funciones
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn import metrics

### **Lectura de datos para modelado**

In [54]:
df_models = pd.read_csv('data//df_employees_models.csv')
df_models.head()

,EmployeeID,Year,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,...,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,retirementType,resignationReason,Attrition
0,1,2015,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,...,0,6,0,3,4,2,3,Not Applicable,Not Applicable,0
1,1,2016,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,...,0,6,0,3,4,2,3,Not Applicable,Not Applicable,0
2,2,2015,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Single,...,1,3,1,3,2,4,2,Resignation,Others,1
3,3,2015,32,Travel_Frequently,Research & Development,17,4,Other,4,Married,...,3,2,0,2,2,1,3,Not Applicable,Not Applicable,0
4,3,2016,32,Travel_Frequently,Research & Development,17,4,Other,4,Married,...,3,2,0,2,2,1,3,Not Applicable,Not Applicable,0


In [55]:
df_models = df_models.sort_values(by='Year')

In [56]:
# Eliminación de 'retirementType' y 'resignationReason' ya que arrojan las misma información que la variable objetivo
df_models.drop('retirementType', axis=1, inplace=True)
df_models.drop('resignationReason', axis=1, inplace=True)

In [57]:
df_models

,EmployeeID,Year,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,...,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,Attrition
0,1,2015,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,...,1,11,0,6,0,3,4,2,3,0
4612,2513,2015,34,Travel_Rarely,Research & Development,2,3,Life Sciences,2,Married,...,1,14,0,5,4,2,3,3,2,0
4610,2512,2015,40,Travel_Rarely,Sales,27,2,Medical,3,Divorced,...,4,18,0,3,0,3,4,3,3,0
4609,2511,2015,46,Travel_Rarely,Research & Development,10,3,Medical,1,Married,...,4,11,1,2,3,4,2,4,3,1
4608,2510,2015,26,Travel_Rarely,Human Resources,2,1,Technical Degree,3,Married,...,0,15,2,3,1,4,2,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,713,2016,27,Travel_Rarely,Research & Development,2,3,Life Sciences,2,Married,...,8,18,0,2,0,3,4,3,3,0
5713,3106,2016,30,Non-Travel,Research & Development,2,4,Life Sciences,3,Married,...,0,15,1,2,7,3,1,3,3,0
3570,1942,2016,35,Travel_Rarely,Research & Development,6,4,Medical,1,Divorced,...,1,19,2,3,10,3,3,2,3,0
1302,712,2016,48,Travel_Rarely,Research & Development,2,1,Life Sciences,2,Single,...,1,20,0,2,0,4,4,3,2,0


In [58]:
df_models.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8820 entries, 0 to 8819
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EmployeeID               8820 non-null   int64 
 1   Year                     8820 non-null   int64 
 2   Age                      8820 non-null   int64 
 3   BusinessTravel           8820 non-null   object
 4   Department               8820 non-null   object
 5   DistanceFromHome         8820 non-null   int64 
 6   Education                8820 non-null   int64 
 7   EducationField           8820 non-null   object
 8   JobLevel                 8820 non-null   int64 
 9   MaritalStatus            8820 non-null   object
 10  MonthlyIncome            8820 non-null   int64 
 11  NumCompaniesWorked       8820 non-null   int64 
 12  PercentSalaryHike        8820 non-null   int64 
 13  StockOptionLevel         8820 non-null   int64 
 14  TrainingTimesLastYear    8820 non-null   int6

In [59]:
# Separación de variables categoricas
list_cat=['BusinessTravel','Department','EducationField','MaritalStatus']

In [60]:
# Separación de variable objetivo y datos de entrenamiento (2015)
y_train = df_models[df_models['Year']==2015]['Attrition']
X_train = df_models[df_models['Year']==2015].drop('Attrition', axis=1)

In [61]:
y_train

0       0
4612    0
4610    0
4609    1
4608    1
       ..
2384    0
2778    0
2619    0
2409    0
2773    1
Name: Attrition, Length: 4410, dtype: int64

In [62]:
X_train.head()

,EmployeeID,Year,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,1,2015,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,131160,1,11,0,6,0,3,4,2,3
4612,2513,2015,34,Travel_Rarely,Research & Development,2,3,Life Sciences,2,Married,53430,1,14,0,5,4,2,3,3,2
4610,2512,2015,40,Travel_Rarely,Sales,27,2,Medical,3,Divorced,102740,4,18,0,3,0,3,4,3,3
4609,2511,2015,46,Travel_Rarely,Research & Development,10,3,Medical,1,Married,44220,4,11,1,2,3,4,2,4,3
4608,2510,2015,26,Travel_Rarely,Human Resources,2,1,Technical Degree,3,Married,65780,0,15,2,3,1,4,2,3,2


In [63]:
# Separación datos de test o datos a utilizar para la predicción de los empleados que se retirarían en 2017
X_test = df_models[df_models['Year']==2016].drop('Attrition', axis=1)

In [64]:
X_test

,EmployeeID,Year,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
7276,3960,2016,46,Travel_Rarely,Sales,3,4,Marketing,1,Divorced,42720,7,13,0,3,2,3,4,3,4
448,243,2016,32,Travel_Rarely,Research & Development,1,1,Life Sciences,1,Married,30220,1,16,1,3,0,1,4,2,3
309,167,2016,57,Travel_Rarely,Research & Development,2,2,Life Sciences,4,Divorced,63060,3,12,1,3,0,1,4,2,3
7260,3951,2016,53,Non-Travel,Research & Development,10,4,Medical,2,Married,20700,4,13,2,2,0,1,3,2,4
3101,1686,2016,49,Travel_Rarely,Sales,21,1,Life Sciences,1,Divorced,40110,1,18,2,2,0,4,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,713,2016,27,Travel_Rarely,Research & Development,2,3,Life Sciences,2,Married,48510,8,18,0,2,0,3,4,3,3
5713,3106,2016,30,Non-Travel,Research & Development,2,4,Life Sciences,3,Married,51310,0,15,1,2,7,3,1,3,3
3570,1942,2016,35,Travel_Rarely,Research & Development,6,4,Medical,1,Divorced,119960,1,19,2,3,10,3,3,2,3
1302,712,2016,48,Travel_Rarely,Research & Development,2,1,Life Sciences,2,Single,28190,1,20,0,2,0,4,4,3,2


In [65]:
# Eliminación de la columna 'Year' dado que ya se realizó la separación
X_train.drop('Year', axis=1, inplace=True)
X_test.drop('Year', axis=1, inplace=True)

In [66]:
X_train

,EmployeeID,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,1,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Married,131160,1,11,0,6,0,3,4,2,3
4612,2513,34,Travel_Rarely,Research & Development,2,3,Life Sciences,2,Married,53430,1,14,0,5,4,2,3,3,2
4610,2512,40,Travel_Rarely,Sales,27,2,Medical,3,Divorced,102740,4,18,0,3,0,3,4,3,3
4609,2511,46,Travel_Rarely,Research & Development,10,3,Medical,1,Married,44220,4,11,1,2,3,4,2,4,3
4608,2510,26,Travel_Rarely,Human Resources,2,1,Technical Degree,3,Married,65780,0,15,2,3,1,4,2,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,1298,56,Travel_Rarely,Human Resources,2,1,Medical,2,Single,26440,4,11,1,2,0,4,4,2,3
2778,1512,51,Travel_Rarely,Research & Development,2,3,Life Sciences,2,Divorced,56750,5,13,0,2,1,4,2,3,2
2619,1425,27,Travel_Frequently,Research & Development,18,2,Medical,1,Single,65160,1,19,1,4,0,4,3,3,3
2409,1311,58,Travel_Rarely,Research & Development,8,4,Life Sciences,2,Single,29720,0,14,0,2,1,4,3,2,4


In [67]:
# Formato dummies de train (2015)
X_train_dummies = pd.get_dummies(X_train, columns=list_cat, drop_first=True)

In [68]:
X_train_dummies

,EmployeeID,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,...,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single
0,1,51,6,2,1,131160,1,11,0,6,...,True,False,True,True,False,False,False,False,True,False
4612,2513,34,2,3,2,53430,1,14,0,5,...,True,True,False,True,False,False,False,False,True,False
4610,2512,40,27,2,3,102740,4,18,0,3,...,True,False,True,False,False,True,False,False,False,False
4609,2511,46,10,3,1,44220,4,11,1,2,...,True,True,False,False,False,True,False,False,True,False
4608,2510,26,2,1,3,65780,0,15,2,3,...,True,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,1298,56,2,1,2,26440,4,11,1,2,...,True,False,False,False,False,True,False,False,False,True
2778,1512,51,2,3,2,56750,5,13,0,2,...,True,True,False,True,False,False,False,False,False,False
2619,1425,27,18,2,1,65160,1,19,1,4,...,False,True,False,False,False,True,False,False,False,True
2409,1311,58,8,4,2,29720,0,14,0,2,...,True,True,False,True,False,False,False,False,False,True


In [69]:
# Convertir False a 0 y True a 1
X_train_dummies = X_train_dummies.replace({True: 1, False: 0})

In [70]:
X_train_dummies.head()

,EmployeeID,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,...,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single
0,1,51,6,2,1,131160,1,11,0,6,...,1,0,1,1,0,0,0,0,1,0
4612,2513,34,2,3,2,53430,1,14,0,5,...,1,1,0,1,0,0,0,0,1,0
4610,2512,40,27,2,3,102740,4,18,0,3,...,1,0,1,0,0,1,0,0,0,0
4609,2511,46,10,3,1,44220,4,11,1,2,...,1,1,0,0,0,1,0,0,1,0
4608,2510,26,2,1,3,65780,0,15,2,3,...,1,0,0,0,0,0,0,1,1,0


In [71]:
# Exclusión de la columna 'EmployeeID'
X1= X_train_dummies.loc[:,~X_train_dummies.columns.isin(['EmployeeID'])]

In [72]:
# Escalado de datos de train (2015)
scaler=StandardScaler()
scaler.fit(X1)
X_std=scaler.transform(X1)
X_train_std=pd.DataFrame(X_std,columns=X1.columns)
X_train_std

,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,YearsSinceLastPromotion,...,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,MaritalStatus_Married,MaritalStatus_Single
0,1.5414,-0.3939,-0.8917,-0.9615,1.4051,-0.6785,-1.1506,-0.9320,2.4834,-0.6791,...,0.6398,-1.3741,1.5152,1.1940,-0.3483,-0.6791,-0.2431,-0.3141,1.0882,-0.6856
1,-0.3202,-0.8875,0.0850,-0.0578,-0.2465,-0.6785,-0.3306,-0.9320,1.7075,0.5626,...,0.6398,0.7278,-0.6600,1.1940,-0.3483,-0.6791,-0.2431,-0.3141,1.0882,-0.6856
2,0.3368,2.1973,-0.8917,0.8459,0.8013,0.5246,0.7627,-0.9320,0.1557,-0.6791,...,0.6398,-1.3741,1.5152,-0.8375,-0.3483,1.4724,-0.2431,-0.3141,-0.9189,-0.6856
3,0.9939,0.0996,0.0850,-0.9615,-0.4422,0.5246,-1.1506,0.2420,-0.6202,0.2521,...,0.6398,0.7278,-0.6600,-0.8375,-0.3483,1.4724,-0.2431,-0.3141,1.0882,-0.6856
4,-1.1962,-0.8875,-1.8684,0.8459,0.0160,-1.0795,-0.0573,1.4160,0.1557,-0.3687,...,0.6398,-1.3741,-0.6600,-0.8375,-0.3483,-0.6791,-0.2431,3.1838,1.0882,-0.6856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,2.0889,-0.8875,-1.8684,-0.0578,-0.8199,0.5246,-1.1506,0.2420,-0.6202,-0.6791,...,0.6398,-1.3741,-0.6600,-0.8375,-0.3483,1.4724,-0.2431,-0.3141,-0.9189,1.4586
4406,1.5414,-0.8875,0.0850,-0.0578,-0.1759,0.9256,-0.6039,-0.9320,-0.6202,-0.3687,...,0.6398,0.7278,-0.6600,1.1940,-0.3483,-0.6791,-0.2431,-0.3141,-0.9189,-0.6856
4407,-1.0867,1.0868,-0.8917,-0.9615,0.0028,-0.6785,1.0360,0.2420,0.9316,-0.6791,...,-1.5629,0.7278,-0.6600,-0.8375,-0.3483,1.4724,-0.2431,-0.3141,-0.9189,1.4586
4408,2.3079,-0.1471,1.0618,-0.0578,-0.7502,-1.0795,-0.3306,-0.9320,-0.6202,-0.3687,...,0.6398,0.7278,-0.6600,1.1940,-0.3483,-0.6791,-0.2431,-0.3141,-0.9189,1.4586


### **Selección de variables**

In [73]:
# Selección de variables utilizando la función 'sel_variables'
mcla = LogisticRegression()
mdtc= DecisionTreeClassifier()
mrfc= RandomForestClassifier()
mgbc=GradientBoostingClassifier()
modelos= [mcla, mdtc, mrfc, mgbc]
var_names=funciones.sel_variables(modelos, X_train_std, y_train, threshold=0.07)
var_names.shape

(22,)

In [74]:
X2=X_train_std[var_names] ### matriz con variables seleccionadas
X2.info()
X_train_std.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                4410 non-null   float64
 1   BusinessTravel_Travel_Frequently   4410 non-null   float64
 2   BusinessTravel_Travel_Rarely       4410 non-null   float64
 3   Department_Research & Development  4410 non-null   float64
 4   Department_Sales                   4410 non-null   float64
 5   DistanceFromHome                   4410 non-null   float64
 6   Education                          4410 non-null   float64
 7   EducationField_Life Sciences       4410 non-null   float64
 8   EducationField_Marketing           4410 non-null   float64
 9   EducationField_Medical             4410 non-null   float64
 10  EducationField_Other               4410 non-null   float64
 11  EducationField_Technical Degree    4410 non-null   float

In [75]:
# Ejecución de modelos y cálculo del accuracy a través de la función 'medir_modelos'
accuracy_df_original=funciones.medir_modelos(modelos,"accuracy",X_train_std,y_train,4) ## base con todas las variables 
accuracy_varsel=funciones.medir_modelos(modelos,"accuracy",X2,y_train,4) ### base con variables seleccionadas

In [76]:
# Visualización de la métrica
pd.set_option('display.float_format', lambda x: '%.4f' % x)
accuracy=pd.concat([accuracy_df_original,accuracy_varsel],axis=1)
accuracy.columns=['rl', 'dt', 'rf', 'gb',
       'rl_Sel', 'dt_sel', 'rf_sel', 'gb_Sel']
accuracy

,rl,dt,rf,gb,rl_Sel,dt_sel,rf_sel,gb_Sel
0,0.8622,0.9438,0.9556,0.8858,0.8622,0.9429,0.9592,0.8885
1,0.8568,0.9411,0.9447,0.8840,0.8577,0.9402,0.9447,0.8821
2,0.8557,0.9828,0.9891,0.8884,0.8557,0.9828,0.9882,0.8857
3,0.8612,0.9601,0.9664,0.8902,0.8612,0.9610,0.9673,0.8829


In [81]:
variables=mrfc.feature_names_in_
importancia = mrfc.feature_importances_
importancia

array([0.12905058, 0.082331  , 0.04289502, 0.039934  , 0.11237247,
       0.05862778, 0.07444844, 0.03563724, 0.0427135 , 0.05344866,
       0.04661209, 0.04842015, 0.04233896, 0.035786  , 0.02084522,
       0.01092346, 0.01481142, 0.01225705, 0.01637292, 0.00813159,
       0.01581584, 0.00669556, 0.00739455, 0.01451327, 0.02762321])

In [82]:
# Peso de cada variable en la predicción
df=pd.DataFrame(variables)
df['coef']=importancia

df.sort_values(by='coef', ascending=False)

,0,coef
0,Age,0.1291
4,MonthlyIncome,0.1124
1,DistanceFromHome,0.0823
6,PercentSalaryHike,0.0744
5,NumCompaniesWorked,0.0586
9,YearsSinceLastPromotion,0.0534
11,JobSatisfaction,0.0484
10,EnvironmentSatisfaction,0.0466
2,Education,0.0429
8,TrainingTimesLastYear,0.0427
